# Simplex method

We are going to illustrate simplex tableau method for linear programing using the following example - WG:
$$\max z = 3 x_1 + 5 x_2$$
s.t.
$$\begin{array}{lll}
x_1 & & \le 4 \\
& 2 x_2 &\le 12 \\
3x_1 & + 2x_2 & \le 18\\
x_1, & x_2 & \ge 0
\end{array}
$$

In [1]:
# import package
import numpy as np
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
import pandas as pd
import warnings
#ignore by message
warnings.filterwarnings("ignore", message="divide by zero encountered in true_divide")

## Pivoting

Pivoting with $(i,j)$-entry means converting the matrix by multiple EROS so that the resulting matrix has all zeros in $j$-column, but one in $(i,j)$-entry. This is the key step in updating basic feasible solution.

In [2]:
#pivoting with (i,j)
def pivoting(A, i, j):
  A[i] = A[i]/A[i,j] #scale to get one in (i,j)
  n_rows, _ = A.shape
  for k in range(n_rows):
    if k==i:
      continue # skip i-row
    A[k] = A[k] - A[i]*A[k,j] # replacement to get zero

## minimum ratio test



In [3]:
def min_ratio_test(ratio_list):
  pivot_row = 0
  min_ratio = np.inf
  for i in range(len(ratio_list)):
    if 0 < ratio_list[i] < min_ratio:
      pivot_row = i+1
      min_ratio = ratio_list[i]
  if min_ratio is np.inf:
    return 0 #no leaving variable
  else:
    return pivot_row

# Pivot column and row

In [4]:
def new_pivot(M_mat): #M is the tableau
  row_n, col_n = M_mat.shape
  row_n = row_n-1 #number of constraints in the augmented form
  col_n = col_n-2 #number of variables in the augmented form
  optimal_test = min(M_mat[0, range(1,col_n+1)])
  
  if optimal_test<0:
    pivot_col = np.argmin(M_mat[0,range(1,col_n+1)])+1
  else:
    print(f'pass the optimal test')
    return 0
  ratio_list = np.divide(M_mat[range(1,row_n+1),-1], M_mat[range(1, row_n+1),pivot_col])

  pivot_row = min_ratio_test(ratio_list)
  if pivot_row is 0:
    print(f'no leaving variable, here is the ratio list {ratio_list}')
    return 0
  else:
    return pivot_row, pivot_col

In [5]:
# input initial tableau
M = np.array([
               [1, -3, -5, 0, 0, 0, 0], 
               [0, 1, 0, 1, 0, 0, 4], 
               [0, 0, 2, 0, 1, 0, 12],
               [0, 3, 2, 0, 0, 1, 18]
               ], dtype = float)
m, n = M.shape
m = m-1
n = n-2
print(f'{m} constraints and {n} variables')

print(pd.DataFrame(M)) # print matrix

3 constraints and 5 variables
     0    1    2    3    4    5     6
0  1.0 -3.0 -5.0  0.0  0.0  0.0   0.0
1  0.0  1.0  0.0  1.0  0.0  0.0   4.0
2  0.0  0.0  2.0  0.0  1.0  0.0  12.0
3  0.0  3.0  2.0  0.0  0.0  1.0  18.0


In [6]:
pivot_n = new_pivot(M)
while pivot_n is not 0:
  print(f'new pivot is {pivot_n}')
  pivoting(M, pivot_n[0], pivot_n[1])
  print(pd.DataFrame(M))
  print(f'=======================')
  pivot_n = new_pivot(M)

new pivot is (2, 2)
     0    1    2    3    4    5     6
0  1.0 -3.0  0.0  0.0  2.5  0.0  30.0
1  0.0  1.0  0.0  1.0  0.0  0.0   4.0
2  0.0  0.0  1.0  0.0  0.5  0.0   6.0
3  0.0  3.0  0.0  0.0 -1.0  1.0   6.0
new pivot is (3, 1)
     0    1    2    3         4         5     6
0  1.0  0.0  0.0  0.0  1.500000  1.000000  36.0
1  0.0  0.0  0.0  1.0  0.333333 -0.333333   2.0
2  0.0  0.0  1.0  0.0  0.500000  0.000000   6.0
3  0.0  1.0  0.0  0.0 -0.333333  0.333333   2.0
pass the optimal test


Since the 0-row has no negative number, the last entry of the first row is the maximum value, which is 36.